In [4]:
# Import necessary libraries
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import numpy as np
import json
import duckdb

In [5]:
# create a connection to a file called 'imdb_reviews.db'
con = duckdb.connect('imdb_reviews.db')

In [6]:
con.execute("DESCRIBE").fetchdf()

,table_name,column_names,column_types,temporary
0,budget_movie_data,"[Release Date, Movie, Production Budget, Domes...","[TIMESTAMP, VARCHAR, VARCHAR, VARCHAR, VARCHAR...",False
1,dummie_director,"[movie, director_1_dum, director_2_dum, direct...","[VARCHAR, UTINYINT, UTINYINT, UTINYINT, UTINYI...",False
2,dummie_writer,"[movie, writer_1_dum, writer_2_dum, writer_3_d...","[VARCHAR, UTINYINT, UTINYINT, UTINYINT, UTINYI...",False
3,imdb_review_test,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
4,imdb_review_train,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
5,imdb_review_validation,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
6,movie_directing,"[movie, director_1, director_2, director_3, di...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
7,movie_writer,"[movie, writer_1, writer_2, writer_3, writer_4...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
8,movielens_information,"[movieId, imdbId, tmdbId, rating, year, title,...","[BIGINT, BIGINT, DOUBLE, DOUBLE, VARCHAR, VARC...",False
9,oscars,"[year_film, year_ceremony, ceremony, category,...","[BIGINT, BIGINT, BIGINT, VARCHAR, VARCHAR, VAR...",False


In [7]:
# Load train data set
df_train = con.execute("""SELECT  imdb_review_train.tconst,
                            imdb_review_train.startYear,
                            imdb_review_train.primaryTitle,
 
                           imdb_review_train.runtimeMinutes,
                           imdb_review_train.numVotes,
                           
                           oscars.year_film,
                           oscars.film,
                           oscars.name,
                           oscars.winner,
    
                           movie_writer.writer_1,
                           movie_writer.writer_2,
                           movie_writer.writer_3,
                           movie_writer.writer_4,
                           movie_writer.writer_5,
                           movie_writer.writer_6,
                           movie_writer.writer_7,
                           movie_writer.writer_8,
                           
                           movie_directing.director_1,
                           movie_directing.director_2,
                           movie_directing.director_3,
                           movie_directing.director_4,
                           movie_directing.director_5,
                           
                           imdb_review_train.label
                           
                           
                    FROM imdb_review_train
                    
                    -- Merge the oscar information
                    LEFT JOIN oscars
                    ON imdb_review_train.startYear = oscars.year_film 
                    AND imdb_review_train.primaryTitle = oscars.film

   
                    -- Merge the writer information
                    LEFT JOIN movie_writer
                    ON imdb_review_train.tconst = movie_writer.movie  
                    
                    -- Merge the director information
                    LEFT JOIN movie_directing
                    ON imdb_review_train.tconst = movie_directing.movie  
                    
                    
                    """).fetchdf()
df_train


,tconst,startYear,primaryTitle,runtimeMinutes,numVotes,year_film,film,name,winner,writer_1,...,writer_5,writer_6,writer_7,writer_8,director_1,director_2,director_3,director_4,director_5,label
0,tt0013257,1922,Häxan,91,13679.0,NaN,NaN,NaN,NaN,nm0159725,...,NaN,NaN,NaN,NaN,nm0159725,NaN,NaN,NaN,NaN,True
1,tt0014341,1923,Our Hospitality,65,10911.0,NaN,NaN,NaN,NaN,nm0369841,...,NaN,NaN,NaN,NaN,nm0090007,nm0000036,NaN,NaN,NaN,True
2,tt0014538,1923,Three Ages,63,4312.0,NaN,NaN,NaN,NaN,nm0115669,...,NaN,NaN,NaN,NaN,nm0000036,nm0166836,NaN,NaN,NaN,True
3,tt0017925,1926,The General,67,87784.0,NaN,NaN,NaN,NaN,nm0000036,...,nm0685904,nm0809548,NaN,NaN,nm0115669,nm0000036,NaN,NaN,NaN,True
4,tt0018192,1927,Napoleon,330,7622.0,NaN,NaN,NaN,NaN,nm0304098,...,NaN,NaN,NaN,NaN,nm0304098,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7941,tt0086907,1984,Antonio Gaudí,72,1557.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nm0856267,NaN,NaN,NaN,NaN,True
7942,tt0117293,1996,Paradise Lost: The Child Murders at Robin Hood...,150,16296.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nm0802501,nm0075666,NaN,NaN,NaN,True
7943,tt0847817,2007,Helvetica,80,7786.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nm1216753,NaN,NaN,NaN,NaN,True
7944,tt4881578,2020,Zappa,129,2561.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,nm0935664,NaN,NaN,NaN,NaN,True


In [8]:
df_train['year_film'].isna().sum()

7943

In [9]:
# create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS oscars_merged_train AS SELECT * FROM df_train")



con.close()

InvalidInputException: Invalid Input Error: Failed to cast value: Unimplemented type for cast (VARCHAR -> NULL)

In [10]:
con.close()